## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from vacation_config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\nancy\Desktop\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,297.93,79,20,few clouds
1,1,Sibolga,ID,1.7427,98.7792,298.30,56,9,clear sky
2,2,Lusambo,CD,-4.9667,23.4500,294.94,91,100,light rain
3,3,Matagami,CA,49.7502,-77.6328,255.21,78,100,overcast clouds
4,4,Shakiso,ET,5.7500,38.9167,286.51,72,86,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 290
What is the maximum temperature you would like for your trip? 300


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,297.93,79,20,few clouds
1,1,Sibolga,ID,1.7427,98.7792,298.30,56,9,clear sky
2,2,Lusambo,CD,-4.9667,23.4500,294.94,91,100,light rain
10,10,Hobart,AU,-42.8794,147.3294,291.07,80,100,light rain
13,13,Kapaa,US,22.0752,-159.3190,299.37,66,0,clear sky
...,...,...,...,...,...,...,...,...,...
688,688,Pala,TD,9.3642,14.9046,298.37,12,12,few clouds
690,690,Los Alamos,US,35.8881,-106.3070,294.98,10,0,clear sky
692,692,Tecoanapa,MX,16.5167,-98.7500,299.38,62,100,overcast clouds
695,695,Santa Maria,BR,-29.6842,-53.8069,290.76,88,0,clear sky


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                8
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Current Description    0
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,297.93,79,20,few clouds
1,1,Sibolga,ID,1.7427,98.7792,298.30,56,9,clear sky
2,2,Lusambo,CD,-4.9667,23.4500,294.94,91,100,light rain
10,10,Hobart,AU,-42.8794,147.3294,291.07,80,100,light rain
13,13,Kapaa,US,22.0752,-159.3190,299.37,66,0,clear sky
...,...,...,...,...,...,...,...,...,...
688,688,Pala,TD,9.3642,14.9046,298.37,12,12,few clouds
690,690,Los Alamos,US,35.8881,-106.3070,294.98,10,0,clear sky
692,692,Tecoanapa,MX,16.5167,-98.7500,299.38,62,100,overcast clouds
695,695,Santa Maria,BR,-29.6842,-53.8069,290.76,88,0,clear sky


In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,297.93,few clouds,-3.8004,102.2655,
1,Sibolga,ID,298.30,clear sky,1.7427,98.7792,
2,Lusambo,CD,294.94,light rain,-4.9667,23.4500,
10,Hobart,AU,291.07,light rain,-42.8794,147.3294,
13,Kapaa,US,299.37,clear sky,22.0752,-159.3190,
17,Buctzotz,MX,296.24,clear sky,21.2000,-88.7833,
18,Alagoa Nova,BR,295.50,moderate rain,-7.0708,-35.7583,
19,Kuala Terengganu,MY,299.12,clear sky,5.3302,103.1408,
22,Esperance,AU,291.06,broken clouds,-33.8667,121.9000,
23,Mataura,NZ,296.27,clear sky,-46.1927,168.8643,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,297.93,few clouds,-3.8004,102.2655,Grage Hotel Bengkulu
1,Sibolga,ID,298.30,clear sky,1.7427,98.7792,Hotel Syariah CN Darussalam
2,Lusambo,CD,294.94,light rain,-4.9667,23.4500,
10,Hobart,AU,291.07,light rain,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
13,Kapaa,US,299.37,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
688,Pala,TD,298.37,few clouds,9.3642,14.9046,Nectar Hôtel de Pala
690,Los Alamos,US,294.98,clear sky,35.8881,-106.3070,Holiday Inn Express & Suites Los Alamos Entrad...
692,Tecoanapa,MX,299.38,overcast clouds,16.5167,-98.7500,Hotel Manglares
695,Santa Maria,BR,290.76,clear sky,-29.6842,-53.8069,Hotel Morotin


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))